In [1]:
import pandas as pd
from exosyspop.populations import BinaryPopulation
from exosyspop.populations import TRILEGAL_BGBinaryPopulation
from exosyspop.populations import KeplerBinaryPopulation, PoissonPlanetPopulation
from exosyspop.populations import PopulationMixture

targets = pd.read_hdf('targets.h5')
bgstars = pd.read_hdf('bgstars.h5')

# Sanitize dtypes of targets DataFrame
for c in targets.columns:
    if targets[c].dtype == object:
        targets.loc[:,c] = targets.loc[:,c].astype(str)


import logging
rootLogger = logging.getLogger()

In [ ]:
ebpop = KeplerBinaryPopulation(targets)
bgpop = TRILEGAL_BGBinaryPopulation(targets, bgstars)
plpop = PoissonPlanetPopulation(targets)
pops = PopulationMixture([ebpop,bgpop,plpop])
pops.train_trap(N=3000)
pops.save('mixture',overwrite=True)

In [4]:
pops = PopulationMixture.load('mixture')

In [5]:
pops.params

{'N_pl': 4.0,
 'Rp_max': 20,
 'Rp_min': 0.75,
 'alpha': -1.6,
 'beta': -0.75,
 'beta_a': 0.8,
 'beta_b': 2.0,
 'fB': 0.4,
 'gamma': 0.3,
 'mu_logp': 2.3979400086720375,
 'period_max': 10000.0,
 'period_min': 1,
 'qmin': 0.1,
 'rho_20': 0.005,
 'rho_5': 0.05,
 'sig_logp': 2.3}

In [6]:
obs = pops.observe()
print(len(obs))
obs.groupby('population').count()

4664


,T14_pri,T14_sec,T23_pri,T23_sec,a,aR,b_pri,b_sec,b_target,d_pri,...,inc,k,n_pri,n_sec,occ,period,phase_sec,target_mag,tra,w
population,,,,,,,,,,,,,,,,,,,,,
BGEB,396,396,396,396,396,396,396,396,396,396,...,396,396,396,396,396,396,396,396,396,396
EB,528,528,528,528,528,528,528,528,528,528,...,528,528,528,528,528,528,528,0,528,528
Planet,3740,3740,3740,3740,3740,3740,3740,3740,3740,3740,...,3740,3740,3740,3740,3740,3740,3740,0,3740,3740


In [7]:
pops['BGEB'].rho_bg(10)

0.023207944168063894

In [8]:
pops.set_params(rho_5=1)
pops['BGEB'].rho_bg(10)

0.17099759466766973

In [9]:
obs = pops.observe()
print(len(obs))
obs.groupby('population').count()

6822


,T14_pri,T14_sec,T23_pri,T23_sec,a,aR,b_pri,b_sec,b_target,d_pri,...,inc,k,n_pri,n_sec,occ,period,phase_sec,target_mag,tra,w
population,,,,,,,,,,,,,,,,,,,,,
BGEB,2635,2635,2635,2635,2635,2635,2635,2635,2635,2635,...,2635,2635,2635,2635,2635,2635,2635,2635,2635,2635
EB,515,515,515,515,515,515,515,515,515,515,...,515,515,515,515,515,515,515,0,515,515
Planet,3672,3672,3672,3672,3672,3672,3672,3672,3672,3672,...,3672,3672,3672,3672,3672,3672,3672,0,3672,3672


In [10]:
pops.reset_params()
obs = pops.observe()
print(len(obs))
obs.groupby('population').count()

4569


,T14_pri,T14_sec,T23_pri,T23_sec,a,aR,b_pri,b_sec,b_target,d_pri,...,inc,k,n_pri,n_sec,occ,period,phase_sec,target_mag,tra,w
population,,,,,,,,,,,,,,,,,,,,,
BGEB,375,375,375,375,375,375,375,375,375,375,...,375,375,375,375,375,375,375,375,375,375
EB,538,538,538,538,538,538,538,538,538,538,...,538,538,538,538,538,538,538,0,538,538
Planet,3656,3656,3656,3656,3656,3656,3656,3656,3656,3656,...,3656,3656,3656,3656,3656,3656,3656,0,3656,3656
